In [4]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
from time import strftime
import plotly.express as px
import glob
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [7]:
# Missing Data from November 8th, 2020 to January 11st, 2021
files = os.path.join("C:\\Users\\aaliy\\OneDrive\\Documents\\GitHub\\energyResearch\\", "bingData*.csv")
files = glob.glob(files)

bingData = pd.concat(map(pd.read_csv, files))
bingData

,START_TIME,END_TIME,BUILDING_ID,TYPE,VALUE
0,2019-01-01 00:10:07,2019-01-01 01:10:07,MH,E,8.669
1,2019-01-01 00:10:07,2019-01-01 01:10:07,LA,E,6.808
2,2019-01-01 00:10:07,2019-01-01 01:10:07,KH,E,6.475
3,2019-01-01 00:10:07,2019-01-01 01:10:07,JH,E,8.623
4,2019-01-01 00:10:07,2019-01-01 01:10:07,HE,E,5.472
...,...,...,...,...,...
115590,2021-12-31 22:10:09,2021-12-31 23:10:09,HSB,E,36.000
115591,2021-12-31 22:10:09,2021-12-31 23:10:09,HSB,H,NaN
115592,2021-12-31 22:10:09,2021-12-31 23:10:09,HD,H,2300.000
115593,2021-12-31 22:10:09,2021-12-31 23:10:09,HD,E,NaN


In [8]:
# DATA CLEANING PROCESS
# Isolate to just Type "E" which refers to Electricity
bingData = bingData[bingData['TYPE'] == 'E']

# Add Time Information to bingData Data Frame
bingData['time'] = pd.to_datetime(bingData['START_TIME'])
bingData['Time'] = pd.to_datetime(bingData['time']).dt.time
bingData['Date'] = pd.to_datetime(bingData['time']).dt.date
bingData['Week'] = bingData['time'].dt.strftime("%U")
bingData['Day'] = bingData['time'].dt.strftime("%j")
bingData['Day'] = pd.to_numeric(bingData['Day'], errors='coerce')
bingData['Weekday'] = bingData['time'].dt.strftime("%w")
bingData['Hour'] = pd.to_datetime(bingData['time']).dt.hour

bingData


C:\Users\aaliy\AppData\Local\Temp/ipykernel_40852/2967112715.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bingData['time'] = pd.to_datetime(bingData['START_TIME'])
C:\Users\aaliy\AppData\Local\Temp/ipykernel_40852/2967112715.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bingData['Time'] = pd.to_datetime(bingData['time']).dt.time
C:\Users\aaliy\AppData\Local\Temp/ipykernel_40852/2967112715.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,START_TIME,END_TIME,BUILDING_ID,TYPE,VALUE,time,Time,Date,Week,Day,Weekday,Hour
0,2019-01-01 00:10:07,2019-01-01 01:10:07,MH,E,8.669,2019-01-01 00:10:07,00:10:07,2019-01-01,00,1,2,0
1,2019-01-01 00:10:07,2019-01-01 01:10:07,LA,E,6.808,2019-01-01 00:10:07,00:10:07,2019-01-01,00,1,2,0
2,2019-01-01 00:10:07,2019-01-01 01:10:07,KH,E,6.475,2019-01-01 00:10:07,00:10:07,2019-01-01,00,1,2,0
3,2019-01-01 00:10:07,2019-01-01 01:10:07,JH,E,8.623,2019-01-01 00:10:07,00:10:07,2019-01-01,00,1,2,0
4,2019-01-01 00:10:07,2019-01-01 01:10:07,HE,E,5.472,2019-01-01 00:10:07,00:10:07,2019-01-01,00,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
115587,2021-12-31 22:10:09,2021-12-31 23:10:09,NH,E,6.750,2021-12-31 22:10:09,22:10:09,2021-12-31,52,365,5,22
115588,2021-12-31 22:10:09,2021-12-31 23:10:09,PH,E,6.700,2021-12-31 22:10:09,22:10:09,2021-12-31,52,365,5,22
115589,2021-12-31 22:10:09,2021-12-31 23:10:09,SA,E,7.220,2021-12-31 22:10:09,22:10:09,2021-12-31,52,365,5,22
115590,2021-12-31 22:10:09,2021-12-31 23:10:09,HSB,E,36.000,2021-12-31 22:10:09,22:10:09,2021-12-31,52,365,5,22


In [10]:
unionData = bingData[bingData['BUILDING_ID'] == 'UU']

LHData = bingData[bingData['BUILDING_ID'] == 'LH']

libData = bingData[bingData['BUILDING_ID'] == 'LH']



#Found Max Demand for Each Day
idx = unionData.groupby(['Date'])['VALUE'].transform(max) == unionData['VALUE']
unionData = unionData[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)
unionData

idx = LHData.groupby(['Date'])['VALUE'].transform(max) == LHData['VALUE']
LHData = LHData[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)
LHData

idx = libData.groupby(['Date'])['VALUE'].transform(max) == libData['VALUE']
libData = libData[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)



#unionData = unionData.drop(unionData.index[650:],0)
#unionData

C:\Users\aaliy\AppData\Local\Temp/ipykernel_40852/2707807708.py:24: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  unionData = unionData.drop(unionData.index[650:],0)


,START_TIME,END_TIME,BUILDING_ID,TYPE,VALUE,time,Time,Date,Week,Day,Weekday,Hour
0,2019-01-01 14:10:02,2019-01-01 15:10:02,UU,E,298.09,2019-01-01 14:10:02,14:10:02,2019-01-01,00,1,2,14
1,2019-01-02 10:10:09,2019-01-02 11:10:09,UU,E,441.87,2019-01-02 10:10:09,10:10:09,2019-01-02,00,2,3,10
2,2019-01-03 10:10:03,2019-01-03 11:10:03,UU,E,455.50,2019-01-03 10:10:03,10:10:03,2019-01-03,00,3,4,10
3,2019-01-04 11:10:10,2019-01-04 12:10:10,UU,E,428.00,2019-01-04 11:10:10,11:10:10,2019-01-04,00,4,5,11
4,2019-01-05 11:10:05,2019-01-05 12:10:05,UU,E,209.54,2019-01-05 11:10:05,11:10:05,2019-01-05,00,5,6,11
...,...,...,...,...,...,...,...,...,...,...,...,...
645,2020-10-07 16:10:04,2020-10-07 17:10:04,UU,E,527.85,2020-10-07 16:10:04,16:10:04,2020-10-07,40,281,3,16
646,2020-10-08 14:10:06,2020-10-08 15:10:06,UU,E,549.66,2020-10-08 14:10:06,14:10:06,2020-10-08,40,282,4,14
647,2020-10-09 12:10:08,2020-10-09 13:10:08,UU,E,552.04,2020-10-09 12:10:08,12:10:08,2020-10-09,40,283,5,12
648,2020-10-10 15:10:03,2020-10-10 16:10:03,UU,E,531.29,2020-10-10 15:10:03,15:10:03,2020-10-10,40,284,6,15


In [ ]:
figure = px.line(x=unionData['Date'],y=unionData['VALUE'], title="University Union Electricity Consumption (2019-2020)",labels={'x': 'Date', 'y':'Electricity Consumption (kWh)'})
figure.write_image("newplot.png", engine = "kaleido")

In [12]:
px.scatter(x=unionData['Date'],y=unionData['VALUE'])


In [ ]:
#How to deal with that one month of missing data? -> Maybe remove 2020 because covid
#do sklearn multivariable regression + calculate error

In [ ]:
#Imported weather Data
weather = pd.read_csv('weatherData2019-2021.csv')
weather = pd.DataFrame({'DateTime':weather['DATE'], 'Temp':weather['HourlyDryBulbTemperature'],'Humidity':weather['HourlyRelativeHumidity']}).dropna()
weather

weather['time'] = pd.to_datetime(weather['DateTime'])
weather['Time'] = pd.to_datetime(weather['time']).dt.time
weather['Date'] = pd.to_datetime(weather['time']).dt.date
weather['Week'] = weather['time'].dt.strftime("%U")
weather['Day'] = weather['time'].dt.strftime("%j")
weather['Day'] = pd.to_numeric(weather['Day'], errors='coerce')
weather['Weekday'] = weather['time'].dt.strftime("%w")
weather['Hour'] = pd.to_datetime(weather['time']).dt.hour

#Found Max Temp for Each Day
idx = weather.groupby(['Date'])['Temp'].transform(max) == weather['Temp']
weather = weather[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)

#Found Max Humidity for Each Day
idx = weather.groupby(['Date'])['Humidity'].transform(max) == weather['Humidity']
weather = weather[idx].drop_duplicates(['Date'], keep='first', ignore_index=True)
weather

C:\Users\aaliy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning:

Columns (42,43,50,52,74,76,90) have mixed types.Specify dtype option on import or set low_memory=False.



,DateTime,Temp,Humidity,time,Time,Date,Week,Day,Weekday,Hour
0,2019-01-01T06:53:00,48.0,80.0,2019-01-01 06:53:00,06:53:00,2019-01-01,00,1,2,6
1,2019-01-02T23:53:00,33.0,89.0,2019-01-02 23:53:00,23:53:00,2019-01-02,00,2,3,23
2,2019-01-03T02:51:00,34.0,93.0,2019-01-03 02:51:00,02:51:00,2019-01-03,00,3,4,2
3,2019-01-04T14:53:00,40.0,65.0,2019-01-04 14:53:00,14:53:00,2019-01-04,00,4,5,14
4,2019-01-05T13:53:00,42.0,76.0,2019-01-05 13:53:00,13:53:00,2019-01-05,00,5,6,13
...,...,...,...,...,...,...,...,...,...,...
1091,2021-12-27T23:53:00,31.0,92.0,2021-12-27 23:53:00,23:53:00,2021-12-27,52,361,1,23
1092,2021-12-28T11:53:00,38.0,70.0,2021-12-28 11:53:00,11:53:00,2021-12-28,52,362,2,11
1093,2021-12-29T20:53:00,38.0,93.0,2021-12-29 20:53:00,20:53:00,2021-12-29,52,363,3,20
1094,2021-12-30T20:53:00,41.0,100.0,2021-12-30 20:53:00,20:53:00,2021-12-30,52,364,4,20


In [ ]:
#MANUAL SOLUTION
#Remove November 8th 2020 to January 11th 2021 (Weather Data index 677-741)
weather_data_adjusted = weather[(weather.index < 677) | (weather.index > 741)]
weather_data_adjusted
#Realized there are other data gaps - maybe try linear interpolation later

#DYNAMIC SOLUTION to remove the data from both data frames if missing data
#Merge johnson and weather data
mergedData = pd.merge(weather, libData, on=['Date'], how='inner')
mergedData

,DateTime,Temp,Humidity,time_x,Time_x,Date,Week_x,Day_x,Weekday_x,Hour_x,...,END_TIME,BUILDING_ID,TYPE,VALUE,time_y,Time_y,Week_y,Day_y,Weekday_y,Hour_y
0,2019-01-01T06:53:00,48.0,80.0,2019-01-01 06:53:00,06:53:00,2019-01-01,00,1,2,6,...,2019-01-01 12:10:01,LH,E,127.471,2019-01-01 11:10:01,11:10:01,00,1,2,11
1,2019-01-02T23:53:00,33.0,89.0,2019-01-02 23:53:00,23:53:00,2019-01-02,00,2,3,23,...,2019-01-02 14:10:04,LH,E,133.110,2019-01-02 13:10:04,13:10:04,00,2,3,13
2,2019-01-03T02:51:00,34.0,93.0,2019-01-03 02:51:00,02:51:00,2019-01-03,00,3,4,2,...,2019-01-03 13:10:03,LH,E,131.252,2019-01-03 12:10:03,12:10:03,00,3,4,12
3,2019-01-04T14:53:00,40.0,65.0,2019-01-04 14:53:00,14:53:00,2019-01-04,00,4,5,14,...,2019-01-04 13:10:10,LH,E,130.482,2019-01-04 12:10:10,12:10:10,00,4,5,12
4,2019-01-05T13:53:00,42.0,76.0,2019-01-05 13:53:00,13:53:00,2019-01-05,00,5,6,13,...,2019-01-05 03:10:02,LH,E,66.718,2019-01-05 02:10:02,02:10:02,00,5,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013,2021-12-27T23:53:00,31.0,92.0,2021-12-27 23:53:00,23:53:00,2021-12-27,52,361,1,23,...,2021-12-28 00:10:09,LH,E,58.443,2021-12-27 23:10:09,23:10:09,52,361,1,23
1014,2021-12-28T11:53:00,38.0,70.0,2021-12-28 11:53:00,11:53:00,2021-12-28,52,362,2,11,...,2021-12-28 01:10:03,LH,E,58.055,2021-12-28 00:10:03,00:10:03,52,362,2,0
1015,2021-12-29T20:53:00,38.0,93.0,2021-12-29 20:53:00,20:53:00,2021-12-29,52,363,3,20,...,2021-12-29 14:10:01,LH,E,59.199,2021-12-29 13:10:01,13:10:01,52,363,3,13
1016,2021-12-30T20:53:00,41.0,100.0,2021-12-30 20:53:00,20:53:00,2021-12-30,52,364,4,20,...,2021-12-30 12:10:05,LH,E,56.524,2021-12-30 11:10:05,11:10:05,52,364,4,11


In [ ]:
#Occupancy value for holidays, weekends, school days

In [ ]:
mergedData['Weekday_y'] = pd.to_numeric(mergedData['Weekday_y'], errors='coerce')
mergedData

,DateTime,Temp,Humidity,time_x,Time_x,Date,Week_x,Day_x,Weekday_x,Hour_x,...,END_TIME,BUILDING_ID,TYPE,VALUE,time_y,Time_y,Week_y,Day_y,Weekday_y,Hour_y
0,2019-01-01T06:53:00,48.0,80.0,2019-01-01 06:53:00,06:53:00,2019-01-01,00,1,2,6,...,2019-01-01 12:10:01,LH,E,127.471,2019-01-01 11:10:01,11:10:01,00,1,2,11
1,2019-01-02T23:53:00,33.0,89.0,2019-01-02 23:53:00,23:53:00,2019-01-02,00,2,3,23,...,2019-01-02 14:10:04,LH,E,133.110,2019-01-02 13:10:04,13:10:04,00,2,3,13
2,2019-01-03T02:51:00,34.0,93.0,2019-01-03 02:51:00,02:51:00,2019-01-03,00,3,4,2,...,2019-01-03 13:10:03,LH,E,131.252,2019-01-03 12:10:03,12:10:03,00,3,4,12
3,2019-01-04T14:53:00,40.0,65.0,2019-01-04 14:53:00,14:53:00,2019-01-04,00,4,5,14,...,2019-01-04 13:10:10,LH,E,130.482,2019-01-04 12:10:10,12:10:10,00,4,5,12
4,2019-01-05T13:53:00,42.0,76.0,2019-01-05 13:53:00,13:53:00,2019-01-05,00,5,6,13,...,2019-01-05 03:10:02,LH,E,66.718,2019-01-05 02:10:02,02:10:02,00,5,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013,2021-12-27T23:53:00,31.0,92.0,2021-12-27 23:53:00,23:53:00,2021-12-27,52,361,1,23,...,2021-12-28 00:10:09,LH,E,58.443,2021-12-27 23:10:09,23:10:09,52,361,1,23
1014,2021-12-28T11:53:00,38.0,70.0,2021-12-28 11:53:00,11:53:00,2021-12-28,52,362,2,11,...,2021-12-28 01:10:03,LH,E,58.055,2021-12-28 00:10:03,00:10:03,52,362,2,0
1015,2021-12-29T20:53:00,38.0,93.0,2021-12-29 20:53:00,20:53:00,2021-12-29,52,363,3,20,...,2021-12-29 14:10:01,LH,E,59.199,2021-12-29 13:10:01,13:10:01,52,363,3,13
1016,2021-12-30T20:53:00,41.0,100.0,2021-12-30 20:53:00,20:53:00,2021-12-30,52,364,4,20,...,2021-12-30 12:10:05,LH,E,56.524,2021-12-30 11:10:05,11:10:05,52,364,4,11


In [ ]:
#US BANK HOLIDAYS
holidays = pd.read_csv('US BANK holidays.csv')
holidays = holidays.drop(columns=['Unnamed: 0']);

mergedData['Date'] = pd.to_datetime(mergedData.Date, format='%Y-%m-%d')
mergedData['Date'] = mergedData['Date'].dt.strftime('%Y-%m-%d')


def holiday(data):

    global add

    add=0
    for x in holidays['date']:

      if (data == holidays.at[add,'date']):
        weekendIndex = 0
        add=0
     #   print("STOP")
        return weekendIndex
      else:
        weekendIndex = 1
        add = add+1
       # print("DROP")

   # print("ROLL")
    add=0
    return weekendIndex

#mergedData['Weekday Index'] = mergedData.apply(lambda x: holiday(x['Date']),axis=1)
mergedData

,DateTime,Temp,Humidity,time_x,Time_x,Date,Week_x,Day_x,Weekday_x,Hour_x,...,END_TIME,BUILDING_ID,TYPE,VALUE,time_y,Time_y,Week_y,Day_y,Weekday_y,Hour_y
0,2019-01-01T06:53:00,48.0,80.0,2019-01-01 06:53:00,06:53:00,2019-01-01,00,1,2,6,...,2019-01-01 12:10:01,LH,E,127.471,2019-01-01 11:10:01,11:10:01,00,1,2,11
1,2019-01-02T23:53:00,33.0,89.0,2019-01-02 23:53:00,23:53:00,2019-01-02,00,2,3,23,...,2019-01-02 14:10:04,LH,E,133.110,2019-01-02 13:10:04,13:10:04,00,2,3,13
2,2019-01-03T02:51:00,34.0,93.0,2019-01-03 02:51:00,02:51:00,2019-01-03,00,3,4,2,...,2019-01-03 13:10:03,LH,E,131.252,2019-01-03 12:10:03,12:10:03,00,3,4,12
3,2019-01-04T14:53:00,40.0,65.0,2019-01-04 14:53:00,14:53:00,2019-01-04,00,4,5,14,...,2019-01-04 13:10:10,LH,E,130.482,2019-01-04 12:10:10,12:10:10,00,4,5,12
4,2019-01-05T13:53:00,42.0,76.0,2019-01-05 13:53:00,13:53:00,2019-01-05,00,5,6,13,...,2019-01-05 03:10:02,LH,E,66.718,2019-01-05 02:10:02,02:10:02,00,5,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013,2021-12-27T23:53:00,31.0,92.0,2021-12-27 23:53:00,23:53:00,2021-12-27,52,361,1,23,...,2021-12-28 00:10:09,LH,E,58.443,2021-12-27 23:10:09,23:10:09,52,361,1,23
1014,2021-12-28T11:53:00,38.0,70.0,2021-12-28 11:53:00,11:53:00,2021-12-28,52,362,2,11,...,2021-12-28 01:10:03,LH,E,58.055,2021-12-28 00:10:03,00:10:03,52,362,2,0
1015,2021-12-29T20:53:00,38.0,93.0,2021-12-29 20:53:00,20:53:00,2021-12-29,52,363,3,20,...,2021-12-29 14:10:01,LH,E,59.199,2021-12-29 13:10:01,13:10:01,52,363,3,13
1016,2021-12-30T20:53:00,41.0,100.0,2021-12-30 20:53:00,20:53:00,2021-12-30,52,364,4,20,...,2021-12-30 12:10:05,LH,E,56.524,2021-12-30 11:10:05,11:10:05,52,364,4,11


In [ ]:
#Create weekday index
#0 for Weekdays being 5 or 6 (weekends) OR if it's a holiday
#1 for anything else
def createWeekdayIndex(weekday, date):
    global add
    add = 0


    for x in holidays['date']:
      if (date == holidays.at[add,'date']):
        weekdayIndex = 0
        add=0
        return weekdayIndex
      else:
        weekdayIndex = 1
        add = add+1
    add=0

    if(weekday == 5) or (weekday == 6):
        weekdayIndex = 0
    return weekdayIndex

mergedData['Weekday Index'] = mergedData.apply(lambda x: createWeekdayIndex(x['Weekday_y'],x['Date']),axis=1)
mergedData


,DateTime,Temp,Humidity,time_x,Time_x,Date,Week_x,Day_x,Weekday_x,Hour_x,...,BUILDING_ID,TYPE,VALUE,time_y,Time_y,Week_y,Day_y,Weekday_y,Hour_y,Weekday Index
0,2019-01-01T06:53:00,48.0,80.0,2019-01-01 06:53:00,06:53:00,2019-01-01,00,1,2,6,...,LH,E,127.471,2019-01-01 11:10:01,11:10:01,00,1,2,11,0
1,2019-01-02T23:53:00,33.0,89.0,2019-01-02 23:53:00,23:53:00,2019-01-02,00,2,3,23,...,LH,E,133.110,2019-01-02 13:10:04,13:10:04,00,2,3,13,1
2,2019-01-03T02:51:00,34.0,93.0,2019-01-03 02:51:00,02:51:00,2019-01-03,00,3,4,2,...,LH,E,131.252,2019-01-03 12:10:03,12:10:03,00,3,4,12,1
3,2019-01-04T14:53:00,40.0,65.0,2019-01-04 14:53:00,14:53:00,2019-01-04,00,4,5,14,...,LH,E,130.482,2019-01-04 12:10:10,12:10:10,00,4,5,12,0
4,2019-01-05T13:53:00,42.0,76.0,2019-01-05 13:53:00,13:53:00,2019-01-05,00,5,6,13,...,LH,E,66.718,2019-01-05 02:10:02,02:10:02,00,5,6,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013,2021-12-27T23:53:00,31.0,92.0,2021-12-27 23:53:00,23:53:00,2021-12-27,52,361,1,23,...,LH,E,58.443,2021-12-27 23:10:09,23:10:09,52,361,1,23,1
1014,2021-12-28T11:53:00,38.0,70.0,2021-12-28 11:53:00,11:53:00,2021-12-28,52,362,2,11,...,LH,E,58.055,2021-12-28 00:10:03,00:10:03,52,362,2,0,1
1015,2021-12-29T20:53:00,38.0,93.0,2021-12-29 20:53:00,20:53:00,2021-12-29,52,363,3,20,...,LH,E,59.199,2021-12-29 13:10:01,13:10:01,52,363,3,13,1
1016,2021-12-30T20:53:00,41.0,100.0,2021-12-30 20:53:00,20:53:00,2021-12-30,52,364,4,20,...,LH,E,56.524,2021-12-30 11:10:05,11:10:05,52,364,4,11,1


In [ ]:
import sklearn
from sklearn.preprocessing import StandardScaler

features = pd.DataFrame({'Weekday Index':mergedData['Weekday Index'], 'Temp':mergedData['Temp'],'Humidity':mergedData['Humidity']}).dropna()
std_scaler = StandardScaler()
std_scaler

features = pd.DataFrame(std_scaler.fit_transform(features), columns=features.columns)

x_train = np.asarray([features['Weekday Index'],features['Temp'], features['Humidity']]).T
y_train = np.asarray(mergedData['VALUE'])


model = LinearRegression()
#calculate beta using train
model.fit(x_train,y_train)
y_pred = model.predict(x_train)


58.3891935749471

In [ ]:
model.coef_
model.score(x_train,y_train) #This is the R^2 value


0.22286949131126244

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
URL = "https://www.binghamton.edu/academics/academic-calendar.html"
#I can use Wayback machine to get earlier snapshots of the data
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")


In [ ]:
results = soup.find(id='academic-calendar')
#Go to Spring -> Go to Month -> Get Date

#Spring 2022
#Summer 2022
#Fall 2022